# $\large{\color{CadetBlue}{\textbf{Biomechanical Features com Decision Tree 🌳}}}$


### Neste dataset, as categorias *Hérnia de Disco* e *Espondilolistese* foram fundidas em uma única categoria rotulada como *‘anormal’*. o objetivo consiste em classificar os pacientes como pertencentes a uma de duas categorias: 

* ### Normal (100 pacientes)
* ### Anormal (210 pacientes)

### Para este desafio, foi decido aplicar o modelo de **Machine Learning Decision Tree**

<a id = "table-of-content"></a>
# $\large{\color{CadetBlue}{\textbf{Sumário 📑}}}$ 

- **[1. Carregando as Bibliotecas 📚](#lib)**
- **[2. Lendo os dados 👀](#ler)**
- **[3. Análise Exploratória dos Dados 🔎](#análise)**
    - [3.1. Proporção das colunas e linhas 📋](#análise1)
    - [3.2. Observando o tipo das Variáveis 🔢](#análise2)
    - [3.3. Verificando se há Valores Nulos ❌](#análise3)
    - [3.4. Separando Variáveis Preditoras e Variável Alvo 🎯](#análise4)
    - [3.5. Proporção dos valores da Variável Alvo 🎯](#análise5)
    - [3.6. Gráfico de Dispersão das Variáveis Preditoras ✨](#análise6)
    - [3.7. HeatMap de Correlação 🗺️🔥](#análise7)
- **[4. Criação do Modelo 🧠](#modelo)**
    - [4.1. Técnica de Validação Cruzada ✔️](#kfold)
    - [4.2. Aplicação do GridSearchCV 🤖](#grid)
    - [4.3. Treinando o Modelo 🏋️](#modelo)
    - [4.4. Visualizando os resultados 🖨️](#print)    
    - [4.5. Obtendo o Melhor Modelo 🏆](#best) 
    - [4.6. Visualizando a Àrvore de Decisão 🌳](#tree) 

    

## $\large{\color{RoyalBlue}{1.}}$ $\large{\color{CadetBlue}{\textbf{Carregando as Bibliotecas 📚}}}$ <a id = "lib"></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import graphviz
import subprocess
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from IPython.display import Image
import os

## $\large{\color{RoyalBlue}{2.}}$ $\large{\color{CadetBlue}{\textbf{Lendo os dados 👀}}}$ <a id = "ler"></a>

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/biomechanical-features-of-orthopedic-patients/column_2C_weka.csv")
df.groupby(['class']).head()

In [ ]:
df.tail()

## $\large{\color{RoyalBlue}{3.}}$ $\large{\color{CadetBlue}{\textbf{Análise Exploratória dos Dados 🔎}}}$ <a id = "análise"></a>

##### $\large{\color{RoyalBlue}{3.1.}}$ $\large{\color{CadetBlue}{\textbf{Proporção das colunas e linhas 📋}}}$ <a id = "análise1"></a>

In [ ]:
df.shape

##### $\large{\color{RoyalBlue}{3.2.}}$ $\large{\color{CadetBlue}{\textbf{Observando o tipo das Variáveis 🔢}}}$ <a id = "análise2"></a>

In [ ]:
df.dtypes

##### $\large{\color{RoyalBlue}{3.3.}}$ $\large{\color{CadetBlue}{\textbf{Verificando se há Valores Nulos ❌}}}$ <a id = "análise3"></a>

In [ ]:
faltantes = (df.isnull().sum()/len(df['pelvic_incidence']))*100
print(faltantes)

### Não dados faltantes!

##### $\large{\color{RoyalBlue}{3.4.}}$ $\large{\color{CadetBlue}{\textbf{Separando Variáveis Preditoras e Variável Alvo 🎯}}}$ <a id = "análise4"></a>

In [ ]:
y = df['class']
x = df.drop('class', axis=1)

##### $\large{\color{RoyalBlue}{3.5.}}$ $\large{\color{CadetBlue}{\textbf{Proporção dos valores da Variável Alvo 🎯}}}$ <a id = "análise5"></a>

In [ ]:
contagem = df['class'].value_counts()
proporcao = (contagem / len(df))*100

print('Quantidade: ', contagem)
print()
print('Porcentagem: ',proporcao)

##### $\large{\color{RoyalBlue}{3.6.}}$ $\large{\color{CadetBlue}{\textbf{Gráfico de Dispersão das Variáveis Preditoras ✨}}}$ <a id = "análise6"></a>

### Tratando-se de 6 variáveis preditoras neste conjunto de dados, todas númericas, resolvi aplicar uma combinação de todos os gráficos, a fim de ver como as váriaveis comportam-se entre si, resultando em 15 gráficos de dispersão, além disso, usei a variável alvo para separar os valores:

In [ ]:
# Lista de todas as combinações possíveis de pares de colunas
column_combinations = list(itertools.combinations(df.columns, 2))

# Número de linhas e colunas para o layout do subplot
num_rows = len(column_combinations) // 3
if len(column_combinations) % 3 != 0:
    num_rows += 1

# Cria um grid de subplots para os gráficos de dispersão
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(10, 15))
fig.subplots_adjust(wspace=0.4, hspace=0.4)

# Cria uma lista de combinações válidas que não inclua 'class' como variável independente
valid_combinations = [(col1, col2) for col1, col2 in column_combinations if col1 != 'class' and col2 != 'class']

# Cria os gráficos de dispersão apenas para combinações válidas
for i, combination in enumerate(valid_combinations):
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    sns.scatterplot(data=df, x=combination[0], y=combination[1], hue='class', palette='Set1', ax=ax)
    ax.set_title(f'{combination[0]} vs {combination[1]}', fontsize=10)

# Ajuste o layout e exiba os gráficos
plt.tight_layout()
plt.show()

### Um ponto interessante de agrupamento foi como todas as variáveis preditoras comportam-se em relação a variável *'degree_spondylolisthesis'*, onde em todos os casos as pessoas normal teve uma concentração maior em baixos níveis desta variável.

### Como o número de pacientes com uma condição anormal é maior que o dobro dos pacientes normal, em muitos gráficos dá uma ilusão de agrupamento diferente entre a condição normal e a anormal, como no caso do *'pelvic_incidence  vs lumbar_lordosis_angle'*, mas a correlação  entre essas variáveis é bem similar entre as duas condições, para esclarecer isto, a seguir foi realizado um heatmap de correlação todas as variáveis preditoras, também agrupadas pela variável alvo, desta forma, temos um gráfico duplo, onde na parte de cima aparece os dados para a condição anormal e embaixo para a normal.

### Neste  exemplo da *'pelvic_incidence  vs lumbar_lordosis_angle'* , foi obtido uma correlação de 0,68 na condição anormal e 0,70 na condição normal, confirmando o que foi dito anteriormente.

##### $\large{\color{RoyalBlue}{3.7.}}$ $\large{\color{CadetBlue}{\textbf{HeatMap de Correlação 🗺️🔥}}}$ <a id = "análise7"></a>

In [ ]:
# Calcule a matriz de correlação
correlation_matrix = df.groupby(['class']).corr()

# Crie um gráfico de matriz de correlação
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()

### O ponto de distinção das classes com o *'degree_spondylolisthesis'* se mantém nesta visualização também, além, ficou claro mais outra variável, a *'pelvic_radius'* tem um nível de correlação muito baixo comparado a todas as variáveis preditoras na condição normal, aparecendo com uma forte linha azul na horizontal e vertical do mapa.

## $\large{\color{RoyalBlue}{4.}}$ $\large{\color{CadetBlue}{\textbf{Criação do Modelo 🧠}}}$ <a id = "modelo"></a>

##### $\large{\color{RoyalBlue}{4.1}}$ $\large{\color{CadetBlue}{\textbf{Técnica de Validação Cruzada ✔️}}}$ <a id = "kfold"></a>

In [ ]:
kfold = KFold(n_splits=5)

modelo = DecisionTreeClassifier()
resultado = cross_val_score(modelo,x,y,cv = kfold)

print('Acurácia: ',resultado.mean())

### Podemos buscar uma acurácia maior, usando o *GridSearchCV* para encontrar os melhores valores de hiperparâmetros.

##### $\large{\color{RoyalBlue}{4.2.}}$ $\large{\color{CadetBlue}{\textbf{Aplicação do GridSearchCV 🤖}}}$ <a id = "grid"></a>

In [ ]:
# Definindo os valores que serão testados em DecisionTree
minimos_split = np.array([2,3,4,5,6,7,8,9,10])
maximo_nivel = np.array([3,4,5,6,7])
algoritmo = ['gini','entropy', 'log_loss']
valores_grid = {'min_samples_split':minimos_split,'max_depth':maximo_nivel,'criterion':algoritmo}

##### $\large{\color{RoyalBlue}{4.3.}}$ $\large{\color{CadetBlue}{\textbf{Treinando o Modelo 🏋️}}}$ <a id = "fit"></a>

In [ ]:
modelo = DecisionTreeClassifier()

grid = GridSearchCV(estimator = modelo, param_grid = valores_grid)
grid.fit(x,y)

##### $\large{\color{RoyalBlue}{4.4.}}$ $\large{\color{CadetBlue}{\textbf{Visualizando os resultados 🖨️}}}$ <a id = "print"></a>

In [ ]:
print('Mínimo split: ', grid.best_estimator_.min_samples_split)
print('Máxima profundidade: ', grid.best_estimator_.max_depth)
print('Algoritmo escolhido: ', grid.best_estimator_.criterion)
print('Acurácia: ', grid.best_score_)

### Deste modo tivemos mais de 10% de melhora nos resultados, é possível através do *best_estimator_* usar o melhor modelo encontrado nesta busca automática e usá-lo para gerar uma visualização da *árvore de decisão*, mas para isso, treinaremos os dados novamente com os valores encontrados.

##### $\large{\color{RoyalBlue}{4.5.}}$ $\large{\color{CadetBlue}{\textbf{Obtendo o Melhor Modelo 🏆}}}$ <a id = "best"></a>

In [ ]:
melhor_modelo = grid.best_estimator_
melhor_modelo.fit(x, y)

##### $\large{\color{RoyalBlue}{4.6.}}$ $\large{\color{CadetBlue}{\textbf{Visualizando a Àrvore de Decisão 🌳}}}$ <a id = "tree"></a>

In [ ]:
# Definir nome do arquivo .dot
arquivo_dot = "/kaggle/working/arvore_decisao.dot"

# Exportar a árvore de decisão para o arquivo .dot
export_graphviz(melhor_modelo, out_file=arquivo_dot, feature_names=list(x.columns), filled=True, rounded=True, class_names=melhor_modelo.classes_)

# Definir o nome do arquivo de saída 
arquivo_png = "/kaggle/working/arvore_decisao.png"

# Comando para converter o arquivo .dot em .png usando o Graphviz
subprocess.run(["dot", "-Tpng", arquivo_dot, "-o", arquivo_png])

# Exibir a imagem diretamente no notebook
Image(filename=arquivo_png)